In [1]:
import os
import json
import pandas as pd
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2,com.datastax.spark:spark-cassandra-connector_2.12:3.1.0 pyspark-shell'

In [2]:
from ast import literal_eval

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("SSKafka") \
    .getOrCreate()
    
# default for startingOffsets is "latest", but "earliest" allows rewind for missed alerts    
dsraw = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "test") \
  .option("startingOffsets", "latest") \
  .load()

In [4]:
### Insert into Cassandra
spark.conf.set("spark.cassandra.connection.host","127.0.0.1:9042")
# spark.conf.set("spark.sql.extensions","com.datastax.spark.connector.CassandraSparkExtensions")
spark.conf.set("spark.sql.catalog.mycatalog", "com.datastax.spark.connector.datasource.CassandraCatalog")

In [5]:
ds = dsraw.selectExpr("CAST(value AS STRING)")

In [6]:
print(type(dsraw))
print(type(ds))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [10]:
alertQuery = ds \
        .writeStream \
        .queryName("qalerts")\
        .format("memory")\
        .start()

IllegalArgumentException: Cannot start query with name qalerts as a query with that name is already active in this SparkSession

In [14]:
alerts = spark.sql("select * from qalerts")
# alerts.tail(15)

In [15]:
alerts.count()

163

In [16]:
df_data = alerts.toPandas()

In [32]:
print(df_data.head()['value'][4])
# df_data.head()

{'post_id': 'P19404418', 'post_type': 'User', 'post_action': 'New', 'post_in_comm': 'COM100002', 'post_msg': 'Illum ullam ipsam odit laborum quasi cupiditate. Praesentium provident consequuntur beatae at deserunt. Atque accusamus ratione fugit facere aperiam placeat porro.', 'post_by': 'U1271664', 'post_likes': 0, 'post_dislikes': 0, 'post_comments': 0, 'post_tstamp': '2021-09-02 22:12:26.662501'}


In [17]:
df5 = df_data.copy()

In [33]:
df_cass_posts = pd.DataFrame()
for i, row in df5.iterrows():
    s = row.value.replace("'", "\"")
    s1 = json.loads(s)
#     print(s1)
    if ('post_id' in s1):
        print ("processing post_id :", s1['post_id'])
        s1['post_tstamp'] = s1['post_tstamp'][:23]
        if (len(df_cass_posts) == 0):
            df_cass_posts = pd.DataFrame.from_records([s1])
        else:
            df_cass_posts = df_cass_posts.append(s1, ignore_index=True)

processing post_id : P19404418
processing post_id : P15263994
processing post_id : P16428820
processing post_id : P18838501
processing post_id : P19404418
processing post_id : P15263994
processing post_id : P16428820
processing post_id : P18838501
processing post_id : P19404418
processing post_id : P15263994
processing post_id : P16428820
processing post_id : P18838501
processing post_id : P19404418
processing post_id : P15263994
processing post_id : P16428820
processing post_id : P18838501
processing post_id : P19404418
processing post_id : P15263994
processing post_id : P16428820
processing post_id : P18838501
processing post_id : P19404418
processing post_id : P15263994
processing post_id : P16428820
processing post_id : P18838501
processing post_id : P19404418
processing post_id : P15263994
processing post_id : P16428820
processing post_id : P18838501
processing post_id : P19404418
processing post_id : P15263994
processing post_id : P16428820
processing post_id : P18838501
processi

In [30]:
df_cass_comm = pd.DataFrame()
for i, row in df5.iterrows():
    s = row.value.replace("'", "\"")
    s1 = json.loads(s)
#     print(s1)
    if ('comm_id' in s1):
        print ("processing comm :", s1['comm_id'])
        s1['comm_tstamp'] = s1['comm_tstamp'][:23]
#         s1['user_comms'] = S1['user_comms']
        if (len(df_cass_comm) == 0):
            df_cass_comm = pd.DataFrame.from_records([s1])
#             pd.Series(df_cass).to_frame()
        else:
            df_cass_comm = df_cass_comm.append(s1, ignore_index=True)

processing comm : COM100002
processing comm : COM100000
processing comm : COM100001


In [18]:
df_cass = pd.DataFrame()
for i, row in df5.iterrows():
    s = row.value.replace("'", "\"")
    s1 = json.loads(s)
#     print(s1)
    if ('user_id' in s1):
        print ("processing user :", s1['user_id'])
        s1['user_created'] = s1['user_created'][:23]
#         s1['user_comms'] = S1['user_comms']
        if (len(df_cass) == 0):
            df_cass = pd.DataFrame.from_records([s1])
#             pd.Series(df_cass).to_frame()
        else:
            df_cass = df_cass.append(s1, ignore_index=True)

processing user : U1271664
processing user : U1361349
processing user : U1314627
processing user : U1446086
processing user : U1450002
processing user : U1602494
processing user : U1841941
processing user : U1402001
processing user : U1610250
processing user : U1140910


In [34]:
df_cass.columns

Index(['user_id', 'user_nm', 'user_city', 'user_st', 'user_interests',
       'user_comms', 'user_comm_cnt', 'user_created'],
      dtype='object')

In [35]:
df_cass.drop_duplicates(subset=['user_id', 'user_nm', 'user_city', 'user_st', 'user_interests',
       'user_comm_cnt', 'user_created'], inplace=True)
df_cass.reset_index(drop=True, inplace=True)

In [42]:
df_cass_comm

,comm_id,comm_nm,comm_users_cnt,comm_users,comm_posts_cnt,comm_posts_id,comm_tstamp
0,COM100002,tech,0,[],0,[],2021-09-02 22:12:26.652
1,COM100000,sports,0,[],0,[],2021-09-02 22:12:26.643
2,COM100001,science,0,[],0,[],2021-09-02 22:12:26.652


In [44]:
df_cass_comm.drop(axis=1, columns=['comm_users','comm_posts_id'], inplace=True)

In [36]:
df_cass

,user_id,user_nm,user_city,user_st,user_interests,user_comms,user_comm_cnt,user_created
0,U1271664,Baiju Chokshi,Malegaon,Punjab,mobiles,COM100002,9,2021-09-02 22:12:26.657
1,U1361349,Emir Hayre,Chapra,Andhra Pradesh,technology,COM100000,9,2021-09-02 22:12:26.694
2,U1314627,Trisha Sundaram,Khammam,Punjab,cricket,COM100000,9,2021-09-02 22:12:27.137
3,U1446086,Saanvi Wable,Coimbatore,Sikkim,badminton,COM100000,9,2021-09-02 22:12:27.405
4,U1450002,Nishith Khare,Sagar,Meghalaya,volleyball,COM100001,9,2021-09-02 22:12:27.538
5,U1602494,Faiyaz Thakkar,Shahjahanpur,Karnataka,mobiles,COM100000,9,2021-09-02 22:12:26.791
6,U1841941,Lakshit Chad,Surat,Punjab,chess,COM100002,9,2021-09-02 22:12:27.027
7,U1402001,Samaira Amble,Hyderabad,Jharkhand,mobiles,COM100001,9,2021-09-02 22:12:27.244
8,U1610250,Sara Raju,Jalandhar,Karnataka,mobiles,COM100000,9,2021-09-02 22:12:26.923
9,U1140910,Ira Bobal,Guwahati,Uttarakhand,space,COM100002,9,2021-09-02 22:12:27.634


In [37]:
df_cass_posts

,post_id,post_type,post_action,post_in_comm,post_msg,post_by,post_likes,post_dislikes,post_comments,post_tstamp
0,P19404418,User,New,COM100002,Illum ullam ipsam odit laborum quasi cupiditat...,U1271664,0,0,0,2021-09-02 22:12:26.662
1,P15263994,User,New,COM100002,Repudiandae ab odio magnam asperiores voluptat...,U1271664,0,0,0,2021-09-02 22:12:26.666
2,P16428820,Comment,New,COM100002,Explicabo similique laboriosam. Maiores numqua...,U1271664,0,0,0,2021-09-02 22:12:26.688
3,P18838501,User,Like,COM100002,Libero voluptatum repellat dolor. Earum repell...,U1271664,0,0,0,2021-09-02 22:12:26.689
4,P19404418,User,New,COM100002,Illum ullam ipsam odit laborum quasi cupiditat...,U1271664,0,0,0,2021-09-02 22:12:26.662
...,...,...,...,...,...,...,...,...,...,...
145,P15751330,Comment,New,COM100002,Temporibus velit dolores temporibus harum even...,U1271664,0,0,0,2021-09-02 22:12:26.693
146,P13728566,Comment,Like,COM100002,Repellendus tenetur cum maxime a earum. Rerum ...,U1271664,0,0,0,2021-09-02 22:12:26.682
147,P15799664,User,Dislike,COM100002,Ea provident eius magni aliquid dicta. Nemo ul...,U1271664,0,0,0,2021-09-02 22:12:26.691
148,P12089350,Comment,Like,COM100002,Quidem cupiditate ex eveniet. Eligendi repelle...,U1271664,0,0,0,2021-09-02 22:12:26.692


In [45]:
sparkDF_cass=spark.createDataFrame(df_cass)
sparkDF_cass_comm=spark.createDataFrame(df_cass_comm)
sparkDF_cass_posts=spark.createDataFrame(df_cass_posts)

In [39]:
sparkDF_cass.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="users", keyspace="raw_stream")\
    .save()

In [46]:
sparkDF_cass_comm.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="community", keyspace="raw_stream")\
    .save()

In [47]:
sparkDF_cass_posts.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="post", keyspace="raw_stream")\
    .save()

In [93]:
# spark.read.format("org.apache.spark.sql.cassandra").options(table="users", keyspace="raw_stream").load().show()
# spark.sql.format

+-------+---+---------+-------------+----------+------------+--------------+-------+-------+
|user_id| id|user_city|user_comm_cnt|user_comms|user_created|user_interests|user_nm|user_st|
+-------+---+---------+-------------+----------+------------+--------------+-------+-------+
+-------+---+---------+-------------+----------+------------+--------------+-------+-------+



In [105]:
# spark.sql("CREATE DATABASE IF NOT EXISTS mycatalog.testks WITH DBPROPERTIES (class='SimpleStrategy',replication_factor='1')")
# spark.sql("CREATE TABLE mycatalog.testks.testtab (key Int, value STRING) USING cassandra PARTITIONED BY (key)")

# # //List their contents
# spark.sql("SHOW NAMESPACES FROM mycatalog").show
# spark.sql("SHOW TABLES FROM mycatalog.testks").show
